### Check Spark Environment

In [1]:
spark

In [2]:
sqlContext

In [3]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

### Load Data

In [4]:
movies = spark.read.load('./data/ml-20m/movies.csv',format='csv',header = True)
ratings = spark.read.load('./data/ml-20m/ratings.csv',format='csv',header = True,inferSchema=True)

In [5]:
movies.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)



In [6]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [7]:
ratings = ratings.select('userId','movieId','rating')
ratings.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      2|   3.5|
|     1|     29|   3.5|
|     1|     32|   3.5|
|     1|     47|   3.5|
|     1|     50|   3.5|
+------+-------+------+
only showing top 5 rows



### Feature Extraction

In [8]:
ratings = ratings.withColumn('userId',ratings['userId'].cast('int'))
ratings = ratings.withColumn('movieId',ratings['movieId'].cast('int'))
ratings = ratings.withColumn('rating',ratings['rating'].cast('float'))
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)



### Train Model

In [9]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

(training, test) = ratings.randomSplit([0.8, 0.2])
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(training)

Py4JJavaError: An error occurred while calling o65.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 10.0 failed 1 times, most recent failure: Lost task 7.0 in stage 10.0 (TID 53, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at scala.collection.mutable.ArrayBuilder$ofFloat.mkArray(ArrayBuilder.scala:471)
	at scala.collection.mutable.ArrayBuilder$ofFloat.result(ArrayBuilder.scala:519)
	at scala.collection.mutable.ArrayBuilder$ofFloat.result(ArrayBuilder.scala:464)
	at org.apache.spark.ml.recommendation.ALS$UncompressedInBlockBuilder.build(ALS.scala:1391)
	at org.apache.spark.ml.recommendation.ALS$$anonfun$23.apply(ALS.scala:1599)
	at org.apache.spark.ml.recommendation.ALS$$anonfun$23.apply(ALS.scala:1593)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$mapValues$1$$anonfun$apply$37$$anonfun$apply$38.apply(PairRDDFunctions.scala:757)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$mapValues$1$$anonfun$apply$37$$an

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

(train,test) = ratings.randomSplit([0.8,0.2])

als = ALS(userCol='userId',itemCol='movieId', ratingCol='rating',coldStartStrategy='drop', nonnegative=True)
param_grid = ParamGridBuilder()
            .addGrid(als.rank,[12,13,14])
            .addGrid(als.maxIter,[18,19,20]).addGrid(als.regParam,[.17,.18,.19])\
             .build()

evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')

cvALS = CrossValidator(estimator=als,estimatorParamMaps=param_grid,evaluator=evaluator,numFolds=5)

In [ ]:
cv = cvALS.fit(ratings)

In [10]:
! java -version

openjdk version "1.8.0_222"
OpenJDK Runtime Environment (build 1.8.0_222-8u222-b10-1ubuntu1~18.04.1-b10)
OpenJDK 64-Bit Server VM (build 25.222-b10, mixed mode)
